# Data Visualization Exam

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import numpy as np

## Data acquisition
CSV file creation from raw data and loading the CSV file into the program.

In [ ]:
dataFolderName = "data"
fileName = dataFolderName + '/DCSC_RACLI_01092021113430630.csv'
df = pd.read_csv(fileName)      # load data from CSV to program
df.head() # data loaded

## Data parsing

In [ ]:
# TODO: Check for errors in data
#       total is equal to the arithmetic mean of the parts? not seem
#       find missing data
#   	check that value apprendista < operaio < dirigente (for territory)

print('\nBefore remove duplicates: ' + str(len(df)) + ' rows')
df.drop_duplicates()
print('After remove duplicates:  ' + str(len(df)) + ' rows')

# are values reasonable?
print('\nMin value is ' + str(df['Value'].min()))
print('Max value is ' + str(df['Value'].max()))

# TODO: Change type

# TODO: Choose the level for hierachical data


# TODO: Transform the data
df['Territorio'] = df['Territorio'].str.replace(' / ','/')

# TODO: rename and translate df fields

## Data filtering

In [ ]:
df2 = df.copy()

# unique data
del df2['TIPO_DATO7'] # always the same (HOUWAG_ENTEMP_AV_MI)
del df2['Tipo dato']  # always the same (Retribuzione lorda oraria per ora retribuita delle posizioni lavorative dipendenti in euro (media).)

# ridondance of information
df2 = df2.drop(['SEXISTAT1', 'ETA1_A','PROFILO_PROF','CLLVT','Seleziona periodo'], axis=1)
# del df2['ATECO_2007']

df2 = df2[df2['Flag Codes'] != 'c'] # delete incomplete data

del df2['Flags']
del df2['Flag Codes']

df2.head()

## Data mining

In [ ]:
# granularity of sectors exists only for entire Italy (no territorial granularity)
df_sectors = df2.query('`Ateco 2007`!="TOTALE"')

# choose granularity of sectors
df_sectors = df_sectors.query('`ATECO_2007`>="A" & `ATECO_2007`<="Z"')

df_sectors = df_sectors.drop(['Territorio', 'ATECO_2007'], axis=1)

In [ ]:
df_territory = df2.query('`Ateco 2007`=="TOTALE"')
df_territory = df_territory.drop(['Ateco 2007', 'ATECO_2007'], axis=1)

In [ ]:
# TODO: start with univariate analysis (one variable at a time), continue with multivariate analysis

## Question 1
Do women earn less than men in Italy in private companies? Where is the most difference?

In [ ]:
df_sex = df_territory.query('Sesso!="totale"')
df_sex = df_sex.drop(['Classe di età','Qualifica contrattuale','Classe di dipendenti'],axis=1)
len(df_sex)

### Plot line chart

In [ ]:
df_sex_tot = df_sex.query('Territorio=="Italia"')

title = 'Gender difference salary in Italy'
source = 'Source: Istat'
labels = ['Men','Women']
colors = ['rgb(0,0,115)', 'rgb(115,0,0)']

x_year = np.arange(df_sex_tot.TIME.min(), df_sex_tot.TIME.max()+1)
x_data = np.vstack((x_year,)*2)

df_sex_tot.sort_values(by='TIME')
y_tot_m = df_sex_tot.query('Sesso=="maschi"')
y_tot_w = df_sex_tot.query('Sesso=="femmine"')
y_data = np.array([y_tot_m['Value'],y_tot_w['Value']])

In [ ]:
fig = go.Figure()

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i], line=dict(color=colors[i]), connectgaps=True ))
    # endpoints
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i],
        mode='markers+text', marker=dict(color=colors[i]),
        text=y_data[i] , textposition="bottom center"))

fig.update_layout(
    xaxis=dict(showline=True, showticklabels=True, ticks='outside',
        linecolor='rgb(204, 204, 204)', linewidth=2, dtick = 1),
    yaxis=dict(showline=True, showticklabels=True, ticks='outside', 
        range = [0, max(df_sex_tot['Value']*1.5)], zeroline=True),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Title
annotations.append(dict(text=title, showarrow=False,
    xref='paper', yref='paper', x=0.0, y=1.05, xanchor='left', yanchor='bottom', 
    font=dict(family='Arial', size=30, color='rgb(37,37,37)'), ))
# Source
annotations.append(dict(text=source,showarrow=False,
    xref='paper', yref='paper', x=0.5, y=-0.1, xanchor='center', yanchor='top', 
    font=dict(family='Arial', size=12, color='rgb(150,150,150)'), ))

# Name of lines
annotations.append(dict(text=labels[0],showarrow=False,
    xref='paper', x=0.3, y=y_data[0,2]+1.5, xanchor='right', yanchor='middle', 
    font=dict(size=16,color=colors[0])))
annotations.append(dict(text=labels[1],showarrow=False,
    xref='paper', x=0.3, y=y_data[1,2]-1.5, xanchor='right', yanchor='middle', 
        font=dict(size=16,color=colors[1])))
# for i in range(0,len(labels)):
#     annotations.append(dict(text=labels[i],showarrow=True,
#         xref='paper', x=0.3, y=y_data[i,2], xanchor='right', yanchor='middle', 
#         font=dict(family='Arial',size=16,color=colors[i])))

fig.update_layout(annotations=annotations)

fig.show()

### Plot gender map

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import geopandas as gpd
from geopandas import GeoDataFrame

In [ ]:
map_df = {}

map_df["map_2014"] = gpd.read_file(f'{dataFolderName}/province_shapes/Prov01012014_g/Prov01012014_g_WGS84.shp')

for year in range(2015,2018):
    fp = f'{dataFolderName}/province_shapes/ProvCM01012017_g/ProvCM01012017_g_WGS84.shp'
    
    map_df[f"map_{year}"] = gpd.read_file(fp) #reading the file stored in variable fp

    map_df[f"map_{year}"].loc[map_df[f"map_{year}"].DEN_PCM=="Aosta","DEN_PCM"] = "Valle d'Aosta/Vallée d'Aoste"
    map_df[f"map_{year}"].loc[map_df[f"map_{year}"].DEN_PCM=="Forli'-Cesena","DEN_PCM"] = "Forlì-Cesena"
    map_df[f"map_{year}"].loc[map_df[f"map_{year}"].DEN_PCM=="Massa Carrara","DEN_PCM"] = "Massa-Carrara"
    map_df[f"map_{year}"].loc[map_df[f"map_{year}"].DEN_PCM=="Bolzano","DEN_PCM"] = "Bolzano/Bozen"

    # print(map_df[f"map_{year}"].plot())
# map_df['map_2015']

In [ ]:
# consider only province data
df_sex_province = df_sex[df_sex['ITTER107'].str.contains('.{5}')]

df_sex_province_year = {}
for year in range(2014,2018):
    temp = df_sex_province.query(f'TIME=={year}')
    temp_f = temp.query('Sesso=="maschi"').set_index('ITTER107').drop('Sesso',axis=1)
    temp_d = temp.query('Sesso=="femmine"').set_index('ITTER107')
    temp_f['Value'] = [temp_f['Value'][prov]-temp_d['Value'][prov] for prov in temp['ITTER107'].drop_duplicates()]
    df_sex_province_year[f'group_{year}'] = temp_f

del temp, temp_d, temp_f
# df_sex_province_year["group_2014"]

In [ ]:
# merge data with map shapes
merged = {}
for year in range(2014,2018):
    merged[f'merged_{year}'] = df_sex_province_year[f'group_{year}'].set_index('Territorio').join(
        map_df[f'map_{year}'].set_index('DEN_PROV' if year==2014 else 'DEN_PCM'))
    # if year==2015: print(merged[f'merged_{year}'][merged[f'merged_{year}'].isnull().any(axis=1)].drop_duplicates())

In [ ]:
vmax = 0
for year in range(2014,2018):
    merged[f'merged_{year}'] = GeoDataFrame(merged[f'merged_{year}'])
    temp = round(merged[f'merged_{year}']['Value'].abs().max())
    if temp > vmax: vmax = temp

vmin, vcenter = -vmax, 0           # set the range for the choropleth

variable = 'Value'
color_map = 'RdBu'

fig, ax = plt.subplots(1, figsize=(10, 6))  # create figure and axes for Matplotlib
ax.axis('off')  # remove the axis
ax.set_title('Gender salary difference', fontdict={'fontsize': '25', 'fontweight' : '3'}) # add a title

# create an annotation for the data source
ax.annotate('Source: Istat', xy=(0.1, .08), xycoords='figure fraction', horizontalalignment='left',
    verticalalignment='top', fontsize=12, color='#555555')

norm = TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)

merged[f'merged_2015'].plot(column=variable, cmap=color_map, linewidth=0.01, ax=ax, edgecolor='0.8', norm=norm)

sm = plt.cm.ScalarMappable(norm=norm, cmap=color_map)
cbar = fig.colorbar(sm)                                     # add the colorbar to the figure

# fig.savefig('map_export.png', dpi=300)        #saving our map as .png file.

## Question 2
In private companies, are salaries higher in northern Italy than in the south? (Where do people earn more? Maybe divide by principal/worker/apprentice)

## Question 3
What are the sectors for which the salaries in private companies are highest in Italy?

In [ ]:
import plotly.graph_objects as go
from ipywidgets import interact

In [ ]:
df_sectors_tot = df_sectors.query('Sesso=="totale" & `Classe di età`=="totale" & \
                              `Classe di dipendenti`=="totale" & `Qualifica contrattuale`=="totale"'
                              )[['Ateco 2007','TIME','Value']]

In [ ]:
# rename sectors in english
it_sec_names = [
    "estrazione di minerali da cave e miniere", "attività manifatturiere", 
    "fornitura di energia elettrica, gas, vapore e aria condizionata",
    "fornitura di acqua reti fognarie, attività di gestione dei rifiuti e risanamento",
    "costruzioni","commercio all'ingrosso e al dettaglio, riparazione di autoveicoli e motocicli",
    "trasporto e magazzinaggio","attività dei servizi di alloggio e di ristorazione",
    "servizi di informazione e comunicazione","attività finanziarie e assicurative",
    "attività immobiliari","attività professionali, scientifiche e tecniche",
    "noleggio, agenzie di viaggio, servizi di supporto alle imprese","istruzione",
    "sanità e assistenza sociale","attività artistiche, sportive, di intrattenimento e divertimento",
    "altre attività di servizi"
    ]
en_sec_names = [
    "extraction of minerals from quarries and mines", "manufacturing activities",
    "supply of electricity, gas, steam and air conditioning",
    "water supply sewerage, waste management and sanitation activities",
    "constructions","wholesale and retail trade, repair of motor vehicles and motorcycles",
    "transport and storage","accommodation and food service activities",
    "information and communication services", "financial and insurance activities",
    "real estate activities", "professional, scientific and technical activities",
    "rental, travel agencies, business support services", "education",
    "health and social work", "arts, sports, entertainment and recreation",
    "other service activities"
    ]
for i in range(0, len(it_sec_names)):
    df_sectors_tot.loc[df_sectors_tot['Ateco 2007']==it_sec_names[i],"Ateco 2007"] = en_sec_names[i]

# df_sectors_tot

In [ ]:
howManyEls=3
df_new = pd.DataFrame(columns=['Ateco 2007','TIME','Value'])

for year in range(2014,2018,1):
  tmp = df_sectors_tot.query(f'TIME=={year}').sort_values(by='Value')
  df_new = df_new.append(tmp.head(howManyEls))
  
  others = {'Ateco 2007':['others'],'TIME':[year],'Value':[round(np.average(tmp.head(-howManyEls).tail(-howManyEls)["Value"]),2)]}
  tmp_others = pd.DataFrame(others,columns=['Ateco 2007','TIME','Value'])
  df_new = df_new.append(tmp_others)
  
  df_new = df_new.append(tmp.tail(howManyEls))

df_new = df_new.sort_values(by='Value').reset_index()

fig = px.bar(df_new, x="Value", y="Ateco 2007", text="Value",
  animation_frame="TIME", range_x=[0,df_new['Value'].max()*1.1])

fig.update_traces(texttemplate='%{text:.2f} ', textposition='inside')

fig.update_layout(
      xaxis=dict( showgrid=False, showline=False ),
      yaxis=dict( showgrid=False, showline=False, ),
      paper_bgcolor='rgb(248, 248, 255)',
      plot_bgcolor='rgb(248, 248, 255)',
    )

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 1000
# fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 1
  
fig.show("notebook")